## Atribuir partidos às emendas

Este notebook carrega `emendas_por_favorecido.csv`, os partidos de deputados
 e os partidos de senadores a partir de dados abertos para associar o partido
 de cada autor de emenda sem alterar o arquivo original.

In [7]:
import pandas as pd
import json
from unidecode import unidecode
from src.senado_party_service import fetch_senator_parties


In [5]:
# Add project root to sys.path so 'src' can be imported
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [8]:
# Baixa e prepara dados de partidos dos senadores
senadores = fetch_senator_parties()
senadores_df = pd.DataFrame(list(senadores.items()),
                            columns=['norm_name', 'siglaPartido'])

In [10]:
# Carrega partidos dos deputados a partir do JSON
with open('../data/deputados_dados_partidos.json') as f:
    deputados = json.load(f)['dados']
deputados_df = pd.DataFrame(deputados)[['nome', 'siglaPartido']]
deputados_df['norm_name'] = deputados_df['nome'].apply(lambda x: unidecode(x).upper())

In [11]:
# Lê emendas e realiza merge sem alterar o CSV original
emendas = pd.read_csv('../data/emendas_por_favorecido.csv')
emendas['norm_name'] = emendas['Nome do Autor da Emenda'].apply(lambda x: unidecode(x).upper())
partidos_df = pd.concat([deputados_df[['norm_name', 'siglaPartido']], senadores_df])
resultado = emendas.merge(partidos_df, on='norm_name', how='left')
resultado.drop(columns='norm_name', inplace=True)

In [13]:
# Salva nova versão sem alterar o arquivo original
resultado.to_csv('../data/emendas_por_favorecido_partidos.csv', index=False)
resultado.head()

,Código da Emenda,Código do Autor da Emenda,Nome do Autor da Emenda,Número da emenda,Tipo de Emenda,Ano/Mês,Código do Favorecido,Favorecido,Natureza Jurídica,Tipo Favorecido,UF Favorecido,Município Favorecido,Valor Recebido,siglaPartido
0,2.024450e+11,4497,PROF. PAULO FERNANDO,5,Emenda Individual - Transferências Especiais,202412,394601000126,DISTRITO FEDERAL,Estado ou Distrito Federal,Pessoa Jurídica,DF,BRASÍLIA,28276676,NaN
1,2.024440e+11,4378,FRED LINHARES,13,Emenda Individual - Transferências Especiais,202412,394601000126,DISTRITO FEDERAL,Estado ou Distrito Federal,Pessoa Jurídica,DF,BRASÍLIA,141093592,REPUBLICANOS
2,2.024440e+11,4449,PROF. REGINALDO VERAS,13,Emenda Individual - Transferências Especiais,202412,394601000126,DISTRITO FEDERAL,Estado ou Distrito Federal,Pessoa Jurídica,DF,BRASÍLIA,214702562,PV
3,2.024420e+11,4166,PASTOR SARGENTO ISIDORIO,1,Emenda Individual - Transferências Especiais,202412,13937032000160,ESTADO DA BAHIA,Estado ou Distrito Federal,Pessoa Jurídica,BA,SALVADOR,813112613,AVANTE
4,2.024420e+11,4190,JOSEILDO RAMOS,10,Emenda Individual - Transferências Especiais,202412,13937032000160,ESTADO DA BAHIA,Estado ou Distrito Federal,Pessoa Jurídica,BA,SALVADOR,130000000,PT


In [15]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import requests, json

def fetch_deputies(year):
    """
    Fetch all deputies for a given year from the Câmara dos Deputados API.
    Returns a dict mapping NOME (uppercased) → SIGLA_PARTIDO.
    """
    url = f"https://dadosabertos.camara.leg.br/api/v2/deputados"
    params = {
        "ano": year,
        "itens": 1000,    # ensure we get all 513 in one go
        "ordenarPor": "nome",
        "ordenar": "ASC"
    }
    resp = requests.get(url, params=params, headers={"Accept":"application/json"})
    resp.raise_for_status()
    data = resp.json().get("dados", [])
    return {d["nome"].upper(): d["siglaPartido"] for d in data}

def fetch_senators():
    """
    Fetch all senators in exercise from the Senado Federal open-data feed.
    Returns a dict mapping NOME (uppercased) → PARTIDO.
    """
    url = "https://www12.senado.leg.br/dados-abertos/conjuntos/senadores-em-exercicio.json"
    resp = requests.get(url)
    resp.raise_for_status()
    data = resp.json().get("List", {}).get("Parlamentares", [])
    mapping = {}
    for item in data:
        nome = item.get("IdentificacaoParlamentar", {}).get("NomeParlamentar", "")
        partido = item.get("IdentificacaoParlamentar", {}).get("SiglaPartido", "")
        if nome:
            mapping[nome.upper()] = partido
    return mapping

def main():
    # 1. Deputies 2023 + 2024 (2024 overrides any repeat from 2023)
    deps23 = fetch_deputies(2023)       # 
    deps24 = fetch_deputies(2024)       # 
    all_deps = {**deps23, **deps24}

    # 2. Senators
    sens = fetch_senators()             # 

    # 3. Combine into one mapping
    full_mapping = {**all_deps, **sens}

    # 4. Write to JSON file
    with open("partidos_2023_2024.json", "w", encoding="utf-8") as f:
        json.dump(full_mapping, f, ensure_ascii=False, indent=2)

    print(f"Generated partidos_2023_2024.json with {len(full_mapping)} entries.")

if __name__ == "__main__":
    main()


HTTPError: 400 Client Error: Bad Request for url: https://dadosabertos.camara.leg.br/api/v2/deputados?ano=2023&itens=1000&ordenarPor=nome&ordenar=ASC